In [1]:
library(broom)
library(sandwich)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Can we use linear regression to estimate risk differences?

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/5/5d/Illustration_of_risk_reduction.svg/1024px-Illustration_of_risk_reduction.svg.png' width='400'>

In [2]:
ee <- 4
en <- 12
ce <- 8
cn <- 8

df <- data.frame(
    treated = c(rep(1, ee + en), rep(0, ce + cn)),
    outcome = c(rep(1, ee), rep(0, en), rep(1, ce), rep(0, cn))
)

rd = ee / (ee + en) - ce / (ce + cn)
std.error = sqrt(ee * en / (ee + en)^3 + ce * cn / (ce + cn)^3)
conf.low = rd - 1.96 * std.error
conf.high = rd + 1.96 * std.error
NNT = 1 / rd

table <- data.frame(experimental = c(ee, en), control = c(ce, cn))
rownames(table) <- c('Outcome', 'No outcome')
table

'Risk difference: {rd}, standard error: {round(std.error, 2)}, 
          95% CI: {round(conf.low, 2)} - {round(conf.high, 2)}, NNT: {NNT}' %>%
    str_glue() %>%
    print

,experimental,control
,<dbl>,<dbl>
Outcome,4,8
No outcome,12,8


Risk difference: -0.25, standard error: 0.17, 
95% CI: -0.57 - 0.07, NNT: -4


In [3]:
lm(outcome ~ treated, data = df)


Call:
lm(formula = outcome ~ treated, data = df)

Coefficients:
(Intercept)      treated  
       0.50        -0.25  


## Uncertainties?

Sandwich robust variance estimator for Gaussian link GLM (OLS).
https://doi.org/10.1093/aje/kwaa044

Implementation example:
http://thestatsgeek.com/2014/02/14/the-robust-sandwich-variance-estimator-for-linear-regression-using-r/

In [4]:
reg <- lm(outcome ~ treated, data = df)

# Heteroskedasticity-consistent estimation of the covariance matrix 
# of the coefficient estimates in regression models.
# Standard errors are square roots of coefficient variances.
sandwich_se <- vcovHC(reg, type = "HC") %>% diag() %>% sqrt()

reg %>%
    tidy %>%
    mutate(
        sandwich.std.error = sandwich_se,
        sandwich.conf.low = estimate - 1.96 * sandwich.std.error,
        sandwich.conf.high = estimate + 1.96 * sandwich.std.error,
        NNT = 1 / estimate
    ) %>%
    filter(term == 'treated')

term,estimate,std.error,statistic,p.value,sandwich.std.error,sandwich.conf.low,sandwich.conf.high,NNT
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
treated,-0.25,0.1707825,-1.46385,0.1536353,0.1653595,-0.5741045,0.07410454,-4


## Can we estimate risk differences using Nelson-Aalen models?